<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/Content_Creation_Crew_with_Web_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai -q
!pip install crewai-tools -q

In [ ]:
!pip install langchain-openai -q

In [1]:
import os
from crewai import Agent, Task, Crew, Process
from textwrap import dedent
# Import the specific ChatOpenAI class to explicitly define the LLM provider
from langchain_openai import ChatOpenAI

# --- Configuration and Environment Setup ---
# CrewAI requires API keys to be available in the os.environ dictionary.

# --- Define the Crew Class ---
class ContentCreationCrew:
    def __init__(self, topic):
        self.topic = topic

        # FIX: Explicitly define the OpenAI LLM instance using ChatOpenAI.
        # This resolves the "LLM Provider NOT provided" error.
        self.openai_llm = ChatOpenAI(model="gpt-4o", temperature=0.7)

    def create_agents(self):
        # 1. Researcher Agent (Conceptual Expert - No Web Search Tool)
        self.researcher = Agent(
            role='Senior Research Analyst',
            goal=f'Synthesize the most critical conceptual data and foundational knowledge about {self.topic}',
            backstory=dedent("""
                You are a highly respected Senior Research Analyst and conceptual expert.
                Your specialty is deeply understanding complex topics and synthesizing
                comprehensive, organized notes from your vast internal knowledge base.
                You generate comprehensive, organized notes that are easy for a writer to use.
            """),
            # No tools are provided, so the agent relies solely on its LLM's knowledge.
            verbose=False,
            allow_delegation=False,
            llm=self.openai_llm # Use the explicit LLM instance
        )

        # 2. Writer Agent
        self.writer = Agent(
            role='Professional Content Writer',
            goal='Write a compelling and engaging 800-word blog post based entirely on the researcher\'s notes.',
            backstory=dedent("""
                You are a famous professional blogger known for transforming complex
                research into clear, engaging, and well-structured articles. You must
                ensure the final output is formatted perfectly in Markdown.
            """),
            verbose=False,
            allow_delegation=False,
            llm=self.openai_llm # Use the explicit LLM instance
        )

    def create_tasks(self):
        # Task 1: Conceptual Research (Internal Knowledge)
        self.research_task = Task(
            description=dedent(f"""
                Conduct a comprehensive conceptual analysis and synthesis of information regarding the topic: '{self.topic}'.
                Identify 3-5 key sub-topics, critical foundational principles,
                and future outlooks based on existing knowledge.

                Your final output MUST be a detailed set of bulleted notes,
                categorized by sub-topic, ready for the writer to use directly.
            """),
            expected_output="A list of detailed, categorized conceptual research notes ready for a writer.",
            agent=self.researcher
        )

        # Task 2: Write the article based on research (This task uses the output of the first task)
        self.writing_task = Task(
            description=dedent(f"""
                Using the research notes provided by the Research Analyst, write a
                compelling and informative blog post about '{self.topic}'.

                The article must be structured with:
                1. A catchy title.
                2. An engaging introduction.
                3. Body sections for each of the sub-topics identified in the notes.
                4. A concluding summary.

                The final output MUST be in complete, polished Markdown format, ready
                for publication, with proper headings (##) and paragraphs.
            """),
            expected_output="A fully formatted, 800-word blog post in clean Markdown.",
            agent=self.writer
        )

    def run(self):
        self.create_agents()
        self.create_tasks()

        # Define the crew with sequential process
        content_crew = Crew(
            agents=[self.researcher, self.writer],
            tasks=[self.research_task, self.writing_task],
            process=Process.sequential,
            verbose=False
        )

        print("### Crew Starting Up ###")

        # Kick off the crew execution
        result = content_crew.kickoff()

        print("\n\n################################################")
        print("### FINAL PUBLICATION READY CONTENT ###")
        print("################################################")
        print(result)

if __name__ == "__main__":
    # Define the topic for the crew to work on
    CREW_TOPIC = "The Future of Quantum Computing in the next 5 years"

    # --- API Key Configuration ---
    # Attempt to retrieve keys using google.colab.userdata, as requested.

    # >>> START: API KEY RETRIEVAL/SETTING LOGIC <<<
    try:
        from google.colab import userdata
        os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
        #print("Keys retrieved using google.colab.userdata and set into os.environ.")
    except ImportError:
        print("google.colab.userdata not available. Ensure OPENAI_API_KEY is set as a standard environment variable.")
        # If not in Colab, the code relies on keys being set externally (e.g., .env file, shell export)
        pass
    # >>> END: API KEY RETRIEVAL/SETTING LOGIC <<<

    is_ready = True

    if not os.getenv("OPENAI_API_KEY"):
        print("FATAL ERROR: OPENAI_API_KEY is not set. Please ensure you have set it in the os.environ dictionary.")
        is_ready = False



    if is_ready:
        # If successfully retrieved, run the crew.
        crew_app = ContentCreationCrew(topic=CREW_TOPIC)
        crew_app.run()
    else:
        print("\nCrew failed to start due to missing API keys.")


### Crew Starting Up ###


################################################
### FINAL PUBLICATION READY CONTENT ###
################################################
```markdown
# The Quantum Leap: Exploring the Future of Quantum Computing in the Next 5 Years

Quantum computing is on the brink of a revolution, poised to redefine the boundaries of technology and science. As we peer into the future, the next five years promise a transformative journey, marked by groundbreaking advancements and global implications. This blog post delves into the anticipated evolution of quantum computing, exploring technological advancements, industry impacts, economic implications, and the challenges that lie ahead.

## Technological Advancements

### Quantum Hardware Improvements

The quest to enhance quantum hardware is relentless, with significant strides expected in qubit quality and quantity. The increase in qubits, while maintaining coherence and reducing error rates, is crucial. Researchers are dev